In [1]:
import numpy as np
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD

c:\Users\49162\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_df = train_gen.flow_from_directory('./data/train', target_size=(150, 150), batch_size=32)
train_df.class_indices

Found 3677 images belonging to 2 classes.


{'bee': 0, 'wasp': 1}

In [3]:
X, y = next(train_df)

In [4]:
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_df = test_gen.flow_from_directory('./data/test', target_size=(150, 150), batch_size=32, shuffle=False)

Found 918 images belonging to 2 classes.


You need to develop the model with following structure:

The shape for input should be (150, 150, 3)

Next, create a convolutional layer (Conv2D):

    Use 32 filters
    Kernel size should be (3, 3) (that's the size of the filter)
    Use 'relu' as activation
    Reduce the size of the feature map with max pooling (MaxPooling2D)
    Set the pooling size to (2, 2)
    Turn the multi-dimensional result into vectors using a Flatten layer

Next, add a Dense layer with 64 neurons and 'relu' activation

Finally, create the Dense layer with 1 neuron - this will be the output

The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use SGD with the following parameters:

SGD(lr=0.002, momentum=0.8)


In [5]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#### Question 1
Since we have a binary classification problem, what is the best loss function for us?

In [6]:
model.compile(loss='binary_crossentropy',
             optimizer=SGD(learning_rate=0.002, momentum=0.8),
             metrics=['acc'])

#### Question 2
What's the number of parameters in the convolutional layer of our model? 

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [9]:
train = ImageDataGenerator(rescale=1./255)
test = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train.flow_from_directory('./data/train',
                                            target_size=(150, 150),
                                            batch_size=20,
                                            class_mode='binary')
test_generator = test.flow_from_directory('./data/test',
                                          target_size=(150, 150),
                                          batch_size=20,
                                        class_mode='binary')

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [11]:
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=test_generator)

Epoch 1/10


184/184 [==============================] - 53s 286ms/step - loss: 0.6696 - acc: 0.5662 - val_loss: 0.6461 - val_acc: 0.5479
Epoch 2/10
184/184 [==============================] - 21s 112ms/step - loss: 0.6266 - acc: 0.6342 - val_loss: 0.5941 - val_acc: 0.6536
Epoch 3/10
184/184 [==============================] - 20s 111ms/step - loss: 0.5767 - acc: 0.7082 - val_loss: 0.6085 - val_acc: 0.6645
Epoch 4/10
184/184 [==============================] - 20s 111ms/step - loss: 0.5349 - acc: 0.7493 - val_loss: 0.5495 - val_acc: 0.7211
Epoch 5/10
184/184 [==============================] - 21s 112ms/step - loss: 0.5079 - acc: 0.7691 - val_loss: 0.5350 - val_acc: 0.7244
Epoch 6/10
184/184 [==============================] - 20s 110ms/step - loss: 0.4793 - acc: 0.7773 - val_loss: 0.5189 - val_acc: 0.7691
Epoch 7/10
184/184 [==============================] - 20s 110ms/step - loss: 0.4588 - acc: 0.7925 - val_loss: 0.5053 - val_acc: 0.7712
Epoch 8/10
184/184 [==============================] -

#### Question 3
What is the median of training accuracy for all the epochs for this model?

In [15]:
np.median(history.history['acc'])

0.7731846570968628

#### Question 4
What is the standard deviation of training loss for all the epochs for this model?

In [18]:
np.std(history.history['loss'])

0.09230264282772438

##### Data Augmentation

Add the following augmentations to your training data generator:

* rotation_range=50,
* width_shift_range=0.1,
* height_shift_range=0.1,
* zoom_range=0.1,
* horizontal_flip=True,
* fill_mode='nearest'


In [20]:
train_aug = ImageDataGenerator(rescale=1./255,
                                   rotation_range=50,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)
train_generator = train_aug.flow_from_directory('./data/train',
                                                target_size=(150, 150),
                                                batch_size=20,
                                                class_mode='binary')

Found 3677 images belonging to 2 classes.


#### Question 5
Let's train our model for 10 more epochs using the same code as previously.

What is the mean of test loss for all the epochs for the model trained with augmentations?

In [21]:
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=test_generator)

Epoch 1/10
184/184 [==============================] - 30s 163ms/step - loss: 0.5031 - acc: 0.7642 - val_loss: 0.5009 - val_acc: 0.7658
Epoch 2/10
184/184 [==============================] - 29s 160ms/step - loss: 0.4849 - acc: 0.7762 - val_loss: 0.4801 - val_acc: 0.7745
Epoch 3/10
184/184 [==============================] - 30s 161ms/step - loss: 0.4746 - acc: 0.7805 - val_loss: 0.4872 - val_acc: 0.7712
Epoch 4/10
184/184 [==============================] - 30s 164ms/step - loss: 0.4757 - acc: 0.7800 - val_loss: 0.5139 - val_acc: 0.7429
Epoch 5/10
184/184 [==============================] - 30s 162ms/step - loss: 0.4769 - acc: 0.7876 - val_loss: 0.4759 - val_acc: 0.7712
Epoch 6/10
184/184 [==============================] - 30s 164ms/step - loss: 0.4678 - acc: 0.7843 - val_loss: 0.4510 - val_acc: 0.7930
Epoch 7/10
184/184 [==============================] - 29s 159ms/step - loss: 0.4643 - acc: 0.7868 - val_loss: 0.5205 - val_acc: 0.7538
Epoch 8/10
184/184 [==============================] - 3

In [24]:
np.mean(history.history['val_loss'])

0.48185963928699493

##### Question 6
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [25]:
np.mean(history.history['val_acc'][6:])

0.7794117480516434